In [1]:
import traci
import os
import sys
import keyboard
import time

# --- Configuration ---
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please declare environment variable 'SUMO_HOME'")

sumoBinary = "sumo-gui"  # Utilisez 'sumo-gui' pour voir la simulation
sumoCmd = [sumoBinary, "-c", "ff_heterogeneous.sumocfg", "--start", "--quit-on-end"]

In [2]:
def run_simulation():
    traci.start(sumoCmd)
    
    # 1. On définit la route une seule fois au début
    traci.route.add("bus_route_2", ["E0E1", "E1E2", "E2E3", "E3E4", "E4E5"])
    
    bus_count = 0
    
    while traci.simulation.getTime() <= 3600:
        traci.simulationStep() # Avancer d'une seconde (ou d'un pas)
        
        sim_time = traci.simulation.getTime()
        
        # 2. Vérifier si on doit ajouter un bus (toutes les 180s à partir de 180s)
        if sim_time >= 180 and sim_time % 180 == 0:
            bus_id = f"bus_2.{bus_count}"
            traci.vehicle.add(bus_id, "bus_route_2", typeID="bus")
            traci.vehicle.setVehicleClass(bus_id, "bus")
            print(f"Bus {bus_id} ajouté à t={sim_time}")
            bus_count += 1
            
        # Optionnel : Sortie manuelle avec la touche 'q'
        if keyboard.is_pressed('q'):
            break
            
    traci.close()

# --- Lancement ---
try:
    run_simulation()
except traci.exceptions.FatalTraCIError:
    print("La simulation a été fermée.")

Bus bus_2.0 ajouté à t=180.0
Bus bus_2.1 ajouté à t=360.0
Bus bus_2.2 ajouté à t=540.0
Bus bus_2.3 ajouté à t=720.0
Bus bus_2.4 ajouté à t=900.0
Bus bus_2.5 ajouté à t=1080.0
Bus bus_2.6 ajouté à t=1260.0
Bus bus_2.7 ajouté à t=1440.0
Bus bus_2.8 ajouté à t=1620.0
Bus bus_2.9 ajouté à t=1800.0
Bus bus_2.10 ajouté à t=1980.0
Bus bus_2.11 ajouté à t=2160.0
Bus bus_2.12 ajouté à t=2340.0
Bus bus_2.13 ajouté à t=2520.0
Bus bus_2.14 ajouté à t=2700.0
Bus bus_2.15 ajouté à t=2880.0
Bus bus_2.16 ajouté à t=3060.0
Bus bus_2.17 ajouté à t=3240.0
Bus bus_2.18 ajouté à t=3420.0
Bus bus_2.19 ajouté à t=3600.0
